In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np

Mounted at /content/drive


# Sentiment, Entity, Page

## Import datasets

In [ ]:
# Load three datasets about content
sentiments = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/Raw Data/sentiments.csv')
entities = pd.read_excel('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/Raw Data/entities.xlsx')
page = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/Raw Data/pages.csv')

In [ ]:
print(page.shape)
print(sentiments.shape)
print(entities.shape)

(20735, 12)
(14298, 9)
(103340, 9)


## Data Preprocessing & Merge 
*   entities_page_sentiments
*   entities_with_sentiments



In [ ]:
# Delete useless columns.
entities.columns

Index(['id', 'page_id', 'begin_offset', 'end_offset', 'score', 'text', 'type',
       'inserted_at', 'updated_at'],
      dtype='object')

In [ ]:
page.columns

Index(['id', 'url', 'project_id', 'published_at', 's3_url', 'title',
       'dom_count', 'image_count', 'link_count', 'scripts_count',
       'inserted_at', 'updated_at'],
      dtype='object')

In [ ]:
page.drop(columns=['s3_url','project_id'],inplace=True)

In [ ]:
sentiments.columns

Index(['id', 'page_id', 'sentiment', 'mixed', 'positive', 'neutral',
       'negative', 'inserted_at', 'updated_at'],
      dtype='object')

In [ ]:
sentiments.drop(columns=['inserted_at','updated_at'],inplace=True)

In [ ]:
page_sentiments = page.merge(sentiments,how='right',left_on='id',right_on='page_id',suffixes=(False,True))

In [ ]:
page_sentiments.head()

,idFalse,url,published_at,title,dom_count,image_count,link_count,scripts_count,inserted_at,updated_at,idTrue,page_id,sentiment,mixed,positive,neutral,negative
0,189e3fa6-cae0-4b7c-ac0a-8ec45d66d8d0,https://byo.com/checkout/order-received/255530/,NaN,Checkout - Brew Your Own,217.0,3.0,44.0,17.0,4/26/20 6:05,4/26/20 6:05,53f73307-b8f6-4e1f-9531-de0ed0220612,189e3fa6-cae0-4b7c-ac0a-8ec45d66d8d0,NEUTRAL,0.000010,0.342735,0.554686,0.102570
1,dafff640-155e-4368-83d3-45a0ca11d601,https://byo.com/checkout/order-received/256628/,NaN,Checkout - Brew Your Own,216.0,3.0,44.0,17.0,5/10/20 11:09,5/10/20 11:09,f894c6ac-850b-4f78-aeee-b1f89a6ffd28,dafff640-155e-4368-83d3-45a0ca11d601,NEUTRAL,0.000010,0.342735,0.554686,0.102570
2,c23e34c1-d2f3-4c63-86ce-1d4cea294ed9,https://byo.com/checkout/order-received/248707/,NaN,Checkout - Brew Your Own,219.0,3.0,44.0,19.0,2/2/20 17:32,2/2/20 17:32,c0631575-8883-4c6a-a843-7592e423f8c4,c23e34c1-d2f3-4c63-86ce-1d4cea294ed9,NEUTRAL,0.000010,0.342735,0.554686,0.102570
3,af7861d8-17db-41e4-a373-aa534c0614ea,https://byo.com/checkout/order-received/255545/,NaN,Checkout - Brew Your Own,217.0,3.0,44.0,17.0,4/26/20 12:29,4/26/20 12:29,695c7074-3980-4ace-8df1-33ab20ec7fc3,af7861d8-17db-41e4-a373-aa534c0614ea,NEUTRAL,0.000010,0.342735,0.554686,0.102570
4,8565ac98-10c6-48ff-9b77-f49fd93d5f74,https://byo.com/article/homebrew-glycol-system...,NaN,Brewbuilt-Icemaster-4-Max - Brew Your Own,436.0,5.0,79.0,40.0,4/7/20 1:09,4/7/20 1:09,aa32e1bd-d9c2-4926-8754-17baba386dba,8565ac98-10c6-48ff-9b77-f49fd93d5f74,NEUTRAL,0.000002,0.000177,0.999794,0.000028


In [ ]:
page_sentiments.columns

Index(['idFalse', 'url', 'published_at', 'title', 'dom_count', 'image_count',
       'link_count', 'scripts_count', 'inserted_at', 'updated_at', 'idTrue',
       'page_id', 'sentiment', 'mixed', 'positive', 'neutral', 'negative'],
      dtype='object')

In [ ]:
page_sentiments = page_sentiments[['url', 'published_at', 'title', 'dom_count', 'image_count',
       'link_count', 'scripts_count', 'inserted_at', 'updated_at', 'page_id', 'sentiment', 'mixed', 'positive', 'neutral', 'negative']]

In [ ]:
page_sentiments.head()

,url,published_at,title,dom_count,image_count,link_count,scripts_count,inserted_at,updated_at,page_id,sentiment,mixed,positive,neutral,negative
0,https://byo.com/checkout/order-received/255530/,NaN,Checkout - Brew Your Own,217.0,3.0,44.0,17.0,4/26/20 6:05,4/26/20 6:05,189e3fa6-cae0-4b7c-ac0a-8ec45d66d8d0,NEUTRAL,0.000010,0.342735,0.554686,0.102570
1,https://byo.com/checkout/order-received/256628/,NaN,Checkout - Brew Your Own,216.0,3.0,44.0,17.0,5/10/20 11:09,5/10/20 11:09,dafff640-155e-4368-83d3-45a0ca11d601,NEUTRAL,0.000010,0.342735,0.554686,0.102570
2,https://byo.com/checkout/order-received/248707/,NaN,Checkout - Brew Your Own,219.0,3.0,44.0,19.0,2/2/20 17:32,2/2/20 17:32,c23e34c1-d2f3-4c63-86ce-1d4cea294ed9,NEUTRAL,0.000010,0.342735,0.554686,0.102570
3,https://byo.com/checkout/order-received/255545/,NaN,Checkout - Brew Your Own,217.0,3.0,44.0,17.0,4/26/20 12:29,4/26/20 12:29,af7861d8-17db-41e4-a373-aa534c0614ea,NEUTRAL,0.000010,0.342735,0.554686,0.102570
4,https://byo.com/article/homebrew-glycol-system...,NaN,Brewbuilt-Icemaster-4-Max - Brew Your Own,436.0,5.0,79.0,40.0,4/7/20 1:09,4/7/20 1:09,8565ac98-10c6-48ff-9b77-f49fd93d5f74,NEUTRAL,0.000002,0.000177,0.999794,0.000028


In [ ]:
page_sentiments.shape

(14298, 15)

In [ ]:
entities.shape

(103340, 9)

In [ ]:
entities.columns

Index(['id', 'page_id', 'begin_offset', 'end_offset', 'score', 'text', 'type',
       'inserted_at', 'updated_at'],
      dtype='object')

In [ ]:
entities.drop(columns=['begin_offset', 'end_offset', 'score'],inplace=True)

In [ ]:
entities_page_sentiments = entities.merge(page_sentiments,on='page_id')

In [ ]:
entities_page_sentiments.head()

,id,page_id,text,type,inserted_at_x,updated_at_x,url,published_at,title,dom_count,image_count,link_count,scripts_count,inserted_at_y,updated_at_y,sentiment,mixed,positive,neutral,negative
0,32e3860f-0512-4a49-ada5-07edc489b6d1,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,29.99,QUANTITY,2019-11-20 03:07:42,2019-11-20 03:07:42,https://byo.com/mr-wizard/late-hop-additions/?...,NaN,Late Hop Additions - Brew Your Own,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595
1,2da9b1df-969b-4871-b89e-e797de8de2a9,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,WizardLate,ORGANIZATION,2019-11-20 03:07:41,2019-11-20 03:07:41,https://byo.com/mr-wizard/late-hop-additions/?...,NaN,Late Hop Additions - Brew Your Own,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595
2,6bf974b6-c95d-438b-b850-548356d01038,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,104 Â°F,QUANTITY,2019-11-20 03:07:41,2019-11-20 03:07:41,https://byo.com/mr-wizard/late-hop-additions/?...,NaN,Late Hop Additions - Brew Your Own,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595
3,c0f31fd1-6c5b-45a6-8d51-9afc854bad23,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,40 Â°C,QUANTITY,2019-11-20 03:07:41,2019-11-20 03:07:41,https://byo.com/mr-wizard/late-hop-additions/?...,NaN,Late Hop Additions - Brew Your Own,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595
4,e77d854e-6713-402b-9271-cb3fce03f3fa,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,176 Â°F,QUANTITY,2019-11-20 03:07:41,2019-11-20 03:07:41,https://byo.com/mr-wizard/late-hop-additions/?...,NaN,Late Hop Additions - Brew Your Own,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595


In [ ]:
entities_page_sentiments.shape

(103283, 20)

In [ ]:
entities_page_sentiments.columns

Index(['id', 'page_id', 'text', 'type', 'inserted_at_x', 'updated_at_x', 'url',
       'published_at', 'title', 'dom_count', 'image_count', 'link_count',
       'scripts_count', 'inserted_at_y', 'updated_at_y', 'sentiment', 'mixed',
       'positive', 'neutral', 'negative'],
      dtype='object')

In [ ]:
entities_with_sentiments = entities_page_sentiments[['id', 'page_id', 'text', 'type', 'title','url',
                                                     'dom_count', 'image_count', 'link_count','scripts_count', 
                                                     'inserted_at_y', 'updated_at_y', 
                                                     'sentiment', 'mixed','positive', 'neutral', 'negative']]

In [ ]:
entities_with_sentiments.head()

,id,page_id,text,type,title,url,dom_count,image_count,link_count,scripts_count,inserted_at_y,updated_at_y,sentiment,mixed,positive,neutral,negative
0,32e3860f-0512-4a49-ada5-07edc489b6d1,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,29.99,QUANTITY,Late Hop Additions - Brew Your Own,https://byo.com/mr-wizard/late-hop-additions/?...,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595
1,2da9b1df-969b-4871-b89e-e797de8de2a9,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,WizardLate,ORGANIZATION,Late Hop Additions - Brew Your Own,https://byo.com/mr-wizard/late-hop-additions/?...,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595
2,6bf974b6-c95d-438b-b850-548356d01038,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,104 Â°F,QUANTITY,Late Hop Additions - Brew Your Own,https://byo.com/mr-wizard/late-hop-additions/?...,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595
3,c0f31fd1-6c5b-45a6-8d51-9afc854bad23,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,40 Â°C,QUANTITY,Late Hop Additions - Brew Your Own,https://byo.com/mr-wizard/late-hop-additions/?...,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595
4,e77d854e-6713-402b-9271-cb3fce03f3fa,3c39a2b1-d169-4a87-96ce-3be16f43dc9f,176 Â°F,QUANTITY,Late Hop Additions - Brew Your Own,https://byo.com/mr-wizard/late-hop-additions/?...,471.0,8.0,84.0,47.0,11/20/19 3:07,11/20/19 3:07,NEUTRAL,0.000082,0.370679,0.592644,0.036595


# Event

### Events data dictionary  
File: `events.csv`  (
Note: This file is pipe (|) delimited, not comma (,) delimited.
Definition: Each row is a discrete action generated by a user. Page loads, clicks
and more can trigger these to be generated.  
Columns:  
- id => unique identifier for each event.  
- current_url => url where the event was generated.  
- device_id => unique identifier for the user that generated the event  
- distinct_id => same as `device_id`  
- initial_referrer => a user's first referrer   
- name => name of the event (i.e. "Purchase Completed")  
- performed_at => time at which the event was generated  
- properties => additional properties (JSON format) that are important for the event (i.e. total_order_price: 50.00)  
- project_id => Can ignore  
- query_params => URL query parameters  
- referrer => the immediate referrer prior to the current_url  
- screen_height => device screen_height  
- screen_width => device screen_width  
- user_agent => browser user_agent for the user  
- inserted_at => time record was inserted in the database  
- updated_at => if an update on the record happens, time update happens


## Import event datasets (Part 1,2,3,4)

In [ ]:
event1 = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/Raw Data/Events/EventsPart1.csv')
event2 = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/Raw Data/Events/EventsPart2.csv')
event3 = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/Raw Data/Events/EventsPart3.csv')
event4 = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/Raw Data/Events/EventsPart4.csv')

In [ ]:
event = pd.concat([event1,event2])
event = pd.concat([event,event3])
event = pd.concat([event,event4])

In [ ]:
event.columns

Index(['id', 'current_url', 'device_id', 'distinct_id', 'initial_referrer',
       'name', 'performed_at', 'properties', 'project_id', 'query_params',
       'referrer', 'screen_height', 'screen_width', 'user_agent',
       'inserted_at', 'updated_at'],
      dtype='object')

In [ ]:
event.shape

(4584206, 16)

## Check and Delete duplicate & useless columns (Part 1)

Keep one as foreign key to merge with Users.csv:
*   device_id vs. **distinct_id**

Delete:   
*   device_id
*   project_id
*   query_params
*   screen_height
*   screen_width
*   inserted_at & updated_at




In [ ]:
event['inserted_at'].isin(event['updated_at']).value_counts() #They are the same thing! & don't give us info

True    4584206
Name: inserted_at, dtype: int64

In [ ]:
event['device_id'].isin(event['distinct_id']).value_counts() #They are the same thing!

True    4584206
Name: device_id, dtype: int64

In [ ]:
# delete these columns
col = ['device_id','project_id','query_params','screen_height','screen_width','updated_at']
#event-----s
events = event.drop(col, axis=1)

# Users

## Load users data & Preprocessing

In [ ]:
users = pd.read_excel('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/Raw Data/users-pipes-all-fields.xlsx')

In [ ]:
# Delete useless columns
users.columns.get_loc('browsers')

28

In [ ]:
users.drop(users.iloc[:,29:],axis=1,inplace=True)

In [ ]:
users.drop(['browsers'],axis=1,inplace=True)

In [ ]:
# Display the duplicate rows 
dup_users = users[users.duplicated()]
print(dup_users.shape)
print("Duplicate Rows except first occurrence based on all columns are :")
dup_users.head()

(1971, 28)
Duplicate Rows except first occurrence based on all columns are :


,distinct_id,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,initial_referrer
230,c25fe1c6-e2cc-4bba-8a57-d619407ba8f5,f,1,0,0,3,22,f,1,0,0,0,32,7,0,14,0,0,0,0,0,0,0,0,0,0,0,https://www.google.com/
734,1e6c289b-e92b-499c-8dfc-a9ac1ff30490,f,0,0,0,0,2,f,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,https://www.google.com/
1776,119a20be-8bce-4753-8944-9e0be9ff8683,f,0,1,0,1,2,f,0,0,0,0,5,1,0,3,0,0,0,1,0,0,0,0,0,0,0,https://www.google.com/
1846,5e07796d-f15e-4416-b810-db2a5e3d0cc3,f,0,0,0,0,3,f,0,0,0,0,4,1,0,3,0,0,0,0,0,0,0,0,0,0,0,https://www.google.com/
2276,9b8f7015-2343-496c-9c48-8f2d8fb0e57e,f,1,9,0,0,1,t,1,0,0,0,14,4,0,6,7,9,0,0,0,0,0,0,0,3,5,https://byo.com/recipes/


In [ ]:
users = users.drop_duplicates().reset_index(drop=True)

In [ ]:
users.shape

(294613, 28)

## Merge Events and Users AS event_user

In [ ]:
events = events[events.distinct_id.isin(users['distinct_id'])]
events.shape

(1242966, 10)

In [ ]:
event_user = events.merge(users, how='left',left_on='distinct_id', right_on='distinct_id', suffixes=(True, False))

In [ ]:
event_user.head()

,id,current_url,distinct_id,initial_referrerTrue,name,performed_at,properties,referrer,user_agent,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,initial_referrerFalse
0,28,https://byo.com/recipe/old-fiddlehead-barleywine/,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,https://www.google.com/,Content Viewed,4:39:00 AM,"{""title"": ""Old Fiddlehead Maple Barleywine"", ""...",https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,https://www.google.com/
1,37,https://byo.com/landing-page/digital-membershi...,adcc583b-c354-4e5d-ba77-21aa57362a57,https://www.google.com/,One Page Checkout,4:39:01 AM,"{""coupon"": """", ""product_id"": ""69709"", ""product...",https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,2019/10/24 6:17:57 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,https://www.google.com/
2,49,https://byo.com/article/maple-beer/,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,https://www.google.com/,Content Viewed,4:39:01 AM,"{""title"": ""Brewing Maple Beer at Home"", ""autho...",https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,https://www.google.com/
3,55,https://byo.com/articles/,adcc583b-c354-4e5d-ba77-21aa57362a57,https://www.google.com/,Search Performed,4:39:01 AM,"{""term"": ""Quad"", ""logged_in"": ""false""}",https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,2019/10/24 6:20:20 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,https://www.google.com/
4,57,https://byo.com/?s=Quad&post_type=article&orde...,adcc583b-c354-4e5d-ba77-21aa57362a57,https://www.google.com/,Search Performed,4:39:01 AM,"{""term"": ""Quad"", ""logged_in"": ""false""}",https://byo.com/articles/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,2019/10/24 6:20:26 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,https://www.google.com/


In [ ]:
# Reorder & rename some columns
event_user = event_user[['id', 'distinct_id', 'current_url', 'name', 'initial_referrerTrue',
       'referrer', 'user_agent', 'performed_at', 'properties','inserted_at',
       'membership_purchased', 'total_sessions', 'content_count',
       'paywalls_count', 'recommended_content_clicks', 'searches_performed',
       'prior_user', 'total_events', 'ecommerce_event_count',
       'account_management_event_count', 'total_cta_count', 'total_ugc_count',
       'homepage_viewed_count', 'product_viewed_count', 'referrer_count',
       'restricted_content_count', 'recipe_view_count', 'mr_wizard_view_count',
       'article_view_count', 'resource_view_count', 'cta_impression_count',
       'cta_conversion_count', 'negative_content_viewed_count',
       'mixed_content_viewed_count', 'positive_content_viewed_count',
       'neutral_content_viewed_count']]

In [ ]:
event_user.rename(columns={"initial_referrerTrue": "initial_referrer","id":"event_id"})

,event_id,distinct_id,current_url,name,initial_referrer,referrer,user_agent,performed_at,properties,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count
0,28,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,https://byo.com/recipe/old-fiddlehead-barleywine/,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,"{""title"": ""Old Fiddlehead Maple Barleywine"", ""...",2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33
1,37,adcc583b-c354-4e5d-ba77-21aa57362a57,https://byo.com/landing-page/digital-membershi...,One Page Checkout,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,"{""coupon"": """", ""product_id"": ""69709"", ""product...",2019/10/24 6:17:57 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0
2,49,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,https://byo.com/article/maple-beer/,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,"{""title"": ""Brewing Maple Beer at Home"", ""autho...",2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33
3,55,adcc583b-c354-4e5d-ba77-21aa57362a57,https://byo.com/articles/,Search Performed,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,"{""term"": ""Quad"", ""logged_in"": ""false""}",2019/10/24 6:20:20 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0
4,57,adcc583b-c354-4e5d-ba77-21aa57362a57,https://byo.com/?s=Quad&post_type=article&orde...,Search Performed,https://www.google.com/,https://byo.com/articles/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,"{""term"": ""Quad"", ""logged_in"": ""false""}",2019/10/24 6:20:26 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242961,86892364,1d70f5d2-e6fe-4ea5-813d-0adfb3ad369f,https://byo.com/article/understanding-malt-spe...,Nano Newsletter Subscribe Toaster,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:8...,3:10:17 PM,"{""action"": ""impression"", ""cta_type"": ""optin"", ...",2021/01/05 3:30:17 AM,f,0,0,0,0,0,t,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1242962,86892435,4931fdc0-20f8-416b-a039-e27b9090b87a,https://byo.com/,Viewed Homepage,https://www.google.com,https://byo.com/my-account/?wcm_redirect_to=po...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3:10:19 PM,{},2021/01/05 3:59:43 AM,f,1,0,0,0,0,f,1,0,0,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0
1242963,86892448,16d0c6ca-3211-48ac-ab4b-ba043649a2ad,https://byo.com/,Viewed Homepage,https://www.google.com,https://www.google.com,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like M...,3:10:19 PM,{},2021/01/05 4:05:08 AM,f,1,0,0,0,0,t,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1242964,86892425,4931fdc0-20f8-416b-a039-e27b9090b87a,https://byo.com/,Viewed Homepage,https://www.google.com,NaN,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3:10:18 PM,{},2021/01/05 3:56:10 AM,f,1,0,0,0,0,f,1,0,0,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Store the dataset for future use
from google.colab import files
event_user.to_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/event_user.csv') 

# Data Merge: Events, Users, Page - event_user_page

In [ ]:
# Merge the event_user data with page data
event_user = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/event_user.csv')

In [ ]:
event_user.shape

(1242966, 37)

In [ ]:
entities_with_sentiments.url.value_counts()

https://byo.com/recipe/stockholm-syndrome-baltic-porter/                          195
https://byo.com/recipe/boulevard-brewing-co-s-tank-7-clone/                       179
https://byo.com/recipe/rock-art-brewerys-vermonster-clone/                        173
https://byo.com/recipe/social-kitchen-and-brewerys-puttin-on-the-spritz-clone/    173
https://byo.com/recipe/gordon-strong-s-american-pale-ale/                         172
                                                                                 ... 
https://byo.com/hite-chocolate-pale-ale                                             1
http://byo.com/recipe/312.html                                                      1
https://byo.com/article/tips-from-pros-tips-from-pros-10/randy-sprecher/            1
https://byo.com/article/yeast-biology/                                              1
http://byo.com/component/resource/article/2437-diacetyl-techniques                  1
Name: url, Length: 5918, dtype: int64

In [ ]:
event_user[~event_user.current_url.isin(page.url)].shape[0]

3457

In [ ]:
event_user = event_user[event_user.current_url.isin(page.url)]

In [ ]:
event_user_page = event_user.merge(page,how='left',left_on='current_url',right_on='url')

In [ ]:
event_user_page.shape

(1239509, 47)

In [ ]:
event_user_page.head()

,Unnamed: 0,id_x,distinct_id,current_url,name,initial_referrerTrue,referrer,user_agent,performed_at,properties,inserted_at_x,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,id_y,url,published_at,title,dom_count,image_count,link_count,scripts_count,inserted_at_y,updated_at
0,0,28,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,https://byo.com/recipe/old-fiddlehead-barleywine/,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,"{""title"": ""Old Fiddlehead Maple Barleywine"", ""...",2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,36ccd600-ff57-4f1e-98ef-5b9804b7f331,https://byo.com/recipe/old-fiddlehead-barleywine/,NaN,Old Fiddlehead Maple Barleywine - Brew Your Own,521.0,9.0,83.0,50.0,9/26/20 19:10,9/26/20 19:10
1,1,37,adcc583b-c354-4e5d-ba77-21aa57362a57,https://byo.com/landing-page/digital-membershi...,One Page Checkout,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,"{""coupon"": """", ""product_id"": ""69709"", ""product...",2019/10/24 6:17:57 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,81aad24f-1bdb-4c96-8552-a1f49f1a4af6,https://byo.com/landing-page/digital-membershi...,NaN,Digital Membership + 14 days Free Trial - Brew...,540.0,16.0,26.0,61.0,12/10/20 18:37,12/10/20 18:37
2,2,49,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,https://byo.com/article/maple-beer/,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,"{""title"": ""Brewing Maple Beer at Home"", ""autho...",2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,6189a121-a840-4644-b045-2bae6280cdfd,https://byo.com/article/maple-beer/,NaN,Brewing Maple Beer at Home - Brew Your Own,609.0,10.0,85.0,49.0,12/11/20 3:32,12/11/20 3:32
3,3,55,adcc583b-c354-4e5d-ba77-21aa57362a57,https://byo.com/articles/,Search Performed,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,"{""term"": ""Quad"", ""logged_in"": ""false""}",2019/10/24 6:20:20 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,d01b6f8d-8b54-406c-9483-fb56cf7374ef,https://byo.com/articles/,NaN,Articles - Brew Your Own,640.0,9.0,150.0,43.0,12/11/20 6:31,12/11/20 6:31
4,4,57,adcc583b-c354-4e5d-ba77-21aa57362a57,https://byo.com/?s=Quad&post_type=article&orde...,Search Performed,https://www.google.com/,https://byo.com/articles/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,"{""term"": ""Quad"", ""logged_in"": ""false""}",2019/10/24 6:20:26 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,8506d0d9-e138-4800-a33e-de8e48024294,https://byo.com/?s=Quad&post_type=article&orde...,NaN,You searched for Quad - Brew Your Own,770.0,11.0,85.0,43.0,10/24/19 18:20,10/24/19 18:20


In [ ]:
event_user_page.to_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/event_user_pages.csv')

In [ ]:
tryon = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/event_user_pages.csv')

In [ ]:
pd.options.display.max_colwidth = 1000

In [ ]:
tryon[['distinct_id','properties']][106:111]

,distinct_id,properties
106,ac39402e-8771-4d65-8b3d-965c56d86f60,"{""title"": ""Saturday, March 28, 2020 Schedule"", ""author"": """", ""logged_in"": ""true"", ""post_name"": ""saturday-march-28-2020-schedule"", ""restricted"": """", ""content_type"": ""bootcamp""}"
107,3f2fa233-444b-4e87-a5c4-0277499c4be4,"{""title"": ""Understanding Malt Spec Sheets"", ""author"": ""Bill Pierce"", ""logged_in"": ""false"", ""post_name"": ""understanding-malt-spec-sheets-advanced-brewing"", ""restricted"": """", ""content_type"": ""article""}"
108,e06b6073-5322-4fb3-a0cb-b57886414e91,"{""title"": ""Kellerbier: The Real Lager of Germany"", ""author"": ""Gordon Strong"", ""logged_in"": ""true"", ""post_name"": ""kellerbier-the-real-lager-of-germany"", ""restricted"": ""1"", ""content_type"": ""article""}"
109,3f2fa233-444b-4e87-a5c4-0277499c4be4,"{""title"": ""The Principles of pH"", ""author"": ""Chris Bible"", ""logged_in"": ""false"", ""post_name"": ""the-principles-of-ph"", ""restricted"": """", ""content_type"": ""article""}"
110,3f2fa233-444b-4e87-a5c4-0277499c4be4,"{""action"": ""impression"", ""cta_type"": ""optin"", ""view_type"": ""slide"", ""campaign_id"": ""xlkywc6dk0rjjh5lgk2p""}"
